In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats as st
import statsmodels.api as sm
import numpy as np

from VaR_func import *
from tests import *

In [21]:
def test_exceeds_vector(vector, alpha=.95):
    return pd.DataFrame({'empiryczne': [vector.mean(),
                                        vector.var(),
                                        stats.binomtest(int(vector.sum()), n=vector.size, p=1-alpha).pvalue,
                                        kupiec(vector, alpha),
                                        christoffersen(vector)],
                         'teoretyczne': [1 - alpha,
                                         alpha*(1-alpha),
                                         '-',
                                         '-',
                                         '-']},
                        index=['średnia', 'wariancja',
                               'binomtest p-wartość',
                               'Kupiec - p-wartość',
                               'christoffersen p-wartość'])

In [28]:
data = pd.read_csv(".\\data\\ropa.csv", index_col='Date', header=0)
data['returns'] = data.interpolate(type='linear').pct_change()
data.index = pd.to_datetime(data.index)
r = list(data['returns'])
r = r[1:]
r_test = r[-298:]
fitted_dist = pd.read_csv('fitted_distributions_best.csv', index_col=0)

In [29]:
dist_type = "genhyperbolic"
filename = './data_distfit/fitted_{}.csv'.format(dist_type)
df1 = pd.read_csv(filename, index_col=0)

In [31]:
print('VaR_95:')
print(kupiec(np.array(I_95), .95))
print(christoffersen(np.array(I_95)))
print(np.mean(scoring_quantile(r_test, v95)))
print('VaR_99:')
print(kupiec(np.array(I_99), .99))
print(christoffersen(np.array(I_99)))
print(np.mean(scoring_quantile(r_test, v99, .099)))

VaR_95:
0.007447710138538399
0.6492634807020252
0.0021482949335459786
VaR_99:
0.18099777733592193
0.9946240358584997
0.052556951236776644


In [41]:
def summary_backtesting(df, pr=True):
    v95 = df1['VaR_95'][1:]
    v99 = df1['VaR_99'][1:]
    ev95 = df1['EVaR_95'][1:]
    ev99 = df1['EVaR_99'][1:]
    I_95 = []
    I_99 = []
    for i in range(len(r_test)):
        I_95.append(v95[i] < r_test[i])
        I_99.append(v99[i] < r_test[i])
    t = '       '
    res = {}
    res['Ku95'] = kupiec(np.array(I_95), .95)
    res['Ku99'] = kupiec(np.array(I_99), .99)
    if sum(I_95) != 0:
        res['Ch95'] = christoffersen(np.array(I_95))
    else: res['Ch95']= None
    if sum(I_99) != 0:
        res['Ch99'] = christoffersen(np.array(I_99))
    else: res['Ch99']= None
    res['Sc95'] = np.mean(scoring_quantile(r_test, v95))
    res['Sc99'] = np.mean(scoring_quantile(r_test, v99))
    res['ESc95'] = np.mean(scoring_expectile(r_test, ev95))
    res['ESc99'] = np.mean(scoring_expectile(r_test, v99))
    
    if pr:
        print('VaR_95:')
        print(t,' Kupiec:', res['Ku95'])
        print(t,'Christ.:', res['Ch95'])
        print(t,'Scoring:', )
        print('VaR_99:')
        print(t,' Kupiec:', res['Ku99'])
        print(t,'Christ.:', res['Ch99'])
        print(t,'Scoring:', np.mean(scoring_quantile(r_test, v99, .099)))
        print('EVaR_95:')
        print(t,'Scoring:', np.mean(scoring_expectile(r_test, ev95,.95)))
        print('EVaR_99:')
        print(t,'Scoring:', np.mean(scoring_expectile(r_test, ev95,.99)))
    return(res)


In [82]:
dst = (
"genhyperbolic" ,
'mielke' ,
"johnsonsu" ,
"nct" ,
#'burr' ,
"genlogistic" ,
"norminvgauss" ,
'burr12' ,
"crystalball" ,
"t" ,
'fisk' ,
"logistic" ,
"tukeylambda" ,
'powernorm' ,
"powerlognorm" ,
"exponweib" ,
"gengamma" ,
"loggamma" ,
"hypsecant" ,
"dweibull" )

results_df=pd.DataFrame()
for dist_type in dst:
    print('')
    print(dist_type)
    filename = './data_distfit/fitted_{}.csv'.format(dist_type)
    df1 = pd.read_csv(filename, index_col=0)
    a = summary_backtesting(df1)
    b =pd.DataFrame.from_dict(a,orient='index',columns=[dist_type])
    results_df = pd.concat([results_df, b.transpose()], ignore_index=False)




genhyperbolic
VaR_95:
         Kupiec: 0.007447710138538399
        Christ.: 0.6492634807020252
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.052556951236776644
EVaR_95:
        Scoring: 0.0001507301877234309
EVaR_99:
        Scoring: 3.053972559021069e-05

mielke
VaR_95:
         Kupiec: 0.0001276037136812569
        Christ.: 0.839492577936688
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.6739419234923615
EVaR_95:
        Scoring: 0.43364530712981836
EVaR_99:
        Scoring: 0.08672963781005456

johnsonsu
VaR_95:
         Kupiec: 0.007447710138538399
        Christ.: 0.6492634807020252
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.05274791301614226
EVaR_95:
        Scoring: 0.00015213245653774416
EVaR_99:
        Scoring: 3.078084711877434e-05

nct
VaR_95:
         Kupiec: 

In [83]:
b = pd.DataFrame.from_dict(a,orient='index',columns=['t'])

In [84]:
results_df

,Ku95,Ku99,Ch95,Ch99,Sc95,Sc99,ESc95,ESc99
genhyperbolic,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002148,0.002927,0.000151,0.000191
mielke,1.276037e-04,1.809978e-01,0.839493,0.994624,0.012401,0.037414,0.433645,0.189369
johnsonsu,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002151,0.002936,0.000152,0.000192
nct,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002151,0.002933,0.000152,0.000191
genlogistic,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002162,0.002984,0.000155,0.000196
norminvgauss,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002154,0.002935,0.000152,0.000192
burr12,1.276037e-04,1.809978e-01,0.839493,0.994624,0.002199,0.002994,0.000163,0.000202
crystalball,6.270683e-04,5.440159e-01,0.774289,0.906913,0.002178,0.002831,0.000145,0.000178
t,1.276037e-04,1.438720e-02,0.839493,NaN,0.002266,0.003341,0.000189,0.000244
fisk,3.219016e-08,1.438720e-02,NaN,NaN,0.047280,0.214725,7.076557,1.511038


In [129]:
wybrane = np.array(("genhyperbolic", "johnsonsu", 't'))

In [130]:
results_df.transpose()[wybrane]

,genhyperbolic,johnsonsu,t
Ku95,0.007448,0.007448,0.000128
Ku99,0.180998,0.180998,0.014387
Ch95,0.649263,0.649263,0.839493
Ch99,0.994624,0.994624,NaN
Sc95,0.002148,0.002151,0.002266
Sc99,0.002927,0.002936,0.003341
ESc95,0.000151,0.000152,0.000189
ESc99,0.000191,0.000192,0.000244


In [131]:
results_df.to_csv('./data/backtesting.csv')

In [120]:
results_df.index

Index(['genhyperbolic', 'mielke', 'johnsonsu', 'nct', 'genlogistic',
       'norminvgauss', 'burr12', 'crystalball', 't', 'fisk', 'logistic',
       'tukeylambda', 'powernorm', 'powerlognorm', 'exponweib', 'gengamma',
       'loggamma', 'hypsecant', 'dweibull'],
      dtype='object')